# Sugestão de tutoriais sobre Pandas:
### https://pandas.pydata.org/docs/user_guide/10min.html
### https://www.kaggle.com/learn/pandas

## Criando dataframe do exemplo da aula

In [2]:
import pandas as pd

data = [['Alice', 'Item 1', 4], ['Alice', 'Item 3', 3], ['Alice', 'Item 4', 4],
      ['Bob', 'Item 1', 1], ['Bob', 'Item 2', 2], ['Bob', 'Item 3', 5], ['Bob', 'Item 5', 3],
      ['Carlos', 'Item 1', 1], ['Carlos', 'Item 4', 5],
      ['Debora', 'Item 2', 3], ['Debora', 'Item 3', 4], ['Debora', 'Item 4', 5], ['Debora', 'Item 5', 3],
      ['Erica', 'Item 1', 2], ['Erica', 'Item 3', 5], ['Erica', 'Item 4', 4], ['Erica', 'Item 5', 5]]

df = pd.DataFrame(data, columns=['user','item', 'rating'])
df

,user,item,rating
0,Alice,Item 1,4
1,Alice,Item 3,3
2,Alice,Item 4,4
3,Bob,Item 1,1
4,Bob,Item 2,2
5,Bob,Item 3,5
6,Bob,Item 5,3
7,Carlos,Item 1,1
8,Carlos,Item 4,5
9,Debora,Item 2,3


## Mapeando usuários e itens para ids

In [3]:
map_users = {user: idx for idx, user in enumerate(df.user.unique())}
map_items = {item: idx for idx, item in enumerate(df.item.unique())}
df['userId'] = df['user'].map(map_users)
df['itemId'] = df['item'].map(map_items)
df

,user,item,rating,userId,itemId
0,Alice,Item 1,4,0,0
1,Alice,Item 3,3,0,1
2,Alice,Item 4,4,0,2
3,Bob,Item 1,1,1,0
4,Bob,Item 2,2,1,3
5,Bob,Item 3,5,1,1
6,Bob,Item 5,3,1,4
7,Carlos,Item 1,1,2,0
8,Carlos,Item 4,5,2,2
9,Debora,Item 2,3,3,3


### Funções para obter informações específicas do DataFrame

In [4]:
# Obter a nota que um usuário deu para um item.
def get_rating(userId,itemId):
    if len(df[(df['userId']==userId)&(df['itemId']==itemId)]) == 0:
        return 0
    return (df.loc[(df.userId==userId) & (df.itemId == itemId),'rating'].iloc[0])

get_rating(1, 5)

0

In [5]:
# Obter a lista de todos os itens que um usuário avaliou.
def get_item_ids(userId):
    if userId not in df['userId'].values:
        return []
    return (df.loc[(df.userId==userId),'itemId'].tolist())

get_item_ids(7)

[]

In [6]:
# Obter o título do item dado o seu id.
def get_item_title(itemId):
    if itemId not in df['itemId'].values:
        return ''
    return (df.loc[(df.itemId == itemId),'item'].iloc[0])

get_item_title(0)

'Item 1'

In [7]:
# Obter a lista de ratings de um usuário.
def get_ratings(userId):
    if userId not in df['userId'].values:
        return []
    return (df.loc[(df.userId==userId),'rating'].tolist())

get_ratings(0)

[4, 3, 4]

In [8]:
import numpy as np

# Obter a média de ratings de um usuário
def get_user_mean(userId):
    return np.mean(get_ratings(userId))

get_user_mean(1)

2.75

### Computar a similaridade de usuários usando Pearson

In [9]:
from statistics import mean
from math import pow, sqrt

def similarity_score(userId1, userId2):
    '''
    userId1 & userId2 : ids dos dois usuários cuja similaridade será computada
    '''
    # Contar quantos itens ambos usuários avaliaram em comum.
    user_list1 = get_item_ids(userId1)
    user_list2 = get_item_ids(userId2)
    common_items = list(set(user_list1) & set(user_list2))
    if len(common_items) == 0:
        return 0

    # Calcular a média de cada usuário
    user1_mean = get_user_mean(userId1)
    user2_mean = get_user_mean(userId2)

    # Cálculo da similaridade.
    sim = []
    norm1 = []
    norm2 = []
    for item in common_items:
        rating1 = get_rating(userId1, item)
        rating2 = get_rating(userId2, item)
        sim.append((rating1 - user1_mean)*(rating2 - user2_mean))
        norm1.append(pow(rating1 - user1_mean, 2))
        norm2.append(pow(rating2 - user2_mean, 2))

    return sum(sim) / (sqrt(sum(norm1)) * sqrt(sum(norm2)))

similarity_score(0, 4)

-0.7302967433402215

### Obter os usuários mais similares

In [10]:
def most_similar_users(userId, k):
    '''
    userId : Targeted User
    k : qtde de vizinhos
    '''
    # Obter lista de usuários.
    user_ids = df.userId.unique().tolist()

    # Obter a similaridade entre o usuário alvo e os demais usuários
    sim = [(similarity_score(userId, u), u) for u in user_ids if u != userId]

    # Ordenação inversa.
    sim.sort()
    sim.reverse()

    # Retornando os usuários mais similares.
    return sim[:k]

print(most_similar_users(0,2))

[(0.263117405792109, 3), (0.0, 2)]


### Calcular a nota

In [11]:
def get_prediction(userId, itemId, k):
    user_mean = get_user_mean(userId)
    similar_users = most_similar_users(userId, k)
    num = []
    den = []

    for s, v in similar_users:
        rv = get_rating(v, itemId)
        if rv == 0:
            continue
        num.append(s * (rv - get_user_mean(v)))
        den.append(s)

    return user_mean + sum(num) / sum(den)

get_prediction(0, 4, 2)

2.9166666666666665